Easy

Будем использовать f1 score

In [51]:
import re
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [59]:
data_path = '../data/singapore_airlines_reviews.csv'

In [63]:
df = pd.read_csv(data_path)

In [64]:
text_processed = df['text'].map(lambda text: re.sub(r'[^\w\s]', '', text))
text_processed = text_processed.map(lambda text: re.sub(r'\n', ' ', text))
y = df['rating']

In [69]:
X_train, X_test, y_train, y_test = train_test_split(text_processed, y, random_state=42)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [70]:
clf = LogisticRegression().fit(X_train, y_train)

In [71]:
y_pred = clf.predict(X_test)
f1_score(y_pred, y_test, average='micro')

0.6592

Medium

In [78]:
def text_process(text, nlp):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\n', ' ', text)
    doc = nlp(text)
    filtered_words = [token.lemma_ for token in doc if not token.is_stop]
    clean_text = ' '.join(filtered_words)
    return clean_text

In [86]:
nlp = spacy.load("en_core_web_sm")
text_processed = df['text'].map(lambda text: text_process(text, nlp))

In [87]:
X_train, X_test, y_train, y_test = train_test_split(text_processed, y, random_state=42)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [94]:
clf_logistic = LogisticRegression().fit(X_train, y_train)
y_pred = clf_logistic.predict(X_test)
f1_score(y_pred, y_test, average='micro')

0.6492

Почему-то в случае логистической регрессии метрика не выросла

Решающее дерево

In [95]:
clf_decision_tree = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = clf_decision_tree.predict(X_test)
f1_score(y_pred, y_test, average='micro')

0.5244

Случайный лес

In [96]:
clf_random_forest = RandomForestClassifier().fit(X_train, y_train)
y_pred = clf_random_forest.predict(X_test)
f1_score(y_pred, y_test, average='micro')

0.5968

Самой лучшей себя показала логистическая регрессия - будем использовать ее

In [102]:
tests = pd.DataFrame(['It was the worst flight of my life. The flight was greatly delayed', 
         'Great flight, great service. We arrived right on time'])
text_processed = tests.map(lambda text: text_process(text, nlp))
tests = vectorizer.transform(text_processed[0])

In [103]:
clf_logistic.predict(tests)

array([1, 5])

Как видно, первый отзыв имеет рейтинг 1, а второй 5, что и соответсвует их текстам.